# 2.1 Advanced Indexing

## Indexing files

As was shown earlier, we can create an index of the data space using the `index()` method:

In [ ]:
import signac
from itertools import islice

project = signac.get_project(root='projects/tutorial')
index = list(project.index())

for doc in index[:3]:
    print(doc)

At this point the index contains information about the statepoint and all data stored in the job document.
If we want to include the `V.txt` text files we used to store data in, with the index, we need to tell **signac** the filename pattern and optionally the file format.
Any name defined as a str constant or even a python class may serve as a format definition.

We will specify that in additon to the job documents, all files matching the regular expression `.*/V\.txt` are to be indexed as `TextFile`.

In [ ]:
definitions = {'.*/V\.txt': 'TextFile'}
index = list(project.index(definitions))
for doc in index[-3:]:
    print(doc)

**Tip**: Consider to to create a shared set of format definitions within your environment which serve as format conventions.

Accessing files via the index is useful, for example to select specific data sub sets.

In [ ]:
import os

def select(doc):
    return 'TextFile' in doc.get('format', '') and doc['statepoint']['p'] < 5.0

docs_selected = [doc for doc in index if select(doc)]
for doc in docs_selected[:3]:
    print('p=', doc['statepoint']['p'], end=' ')
    fn = os.path.join(doc['root'], doc['filename'])
    with open(fn) as file:
        print('V=', file.read().strip())

## Customized Project Crawlers

The `index()` function as well as the `$ signac index` command internally creater a `Crawler` instance to crawl through the data space and create the index.
To have more control over the indexing process, we can do this explicitly:

In [ ]:
from signac.contrib import SignacProjectCrawler

# Specialize a SignacProject Crawler...
class TutorialProjectCrawler(SignacProjectCrawler):
    pass

# Define files to index...
TutorialProjectCrawler.define('.*/V\.txt', 'TextFile')

# Create a crawler instance and generate the index.
crawler = TutorialProjectCrawler(root=project.workspace())
index = list(crawler.crawl())
for doc in index[:3]:
    print(doc)

We could specialize the `IdealGasCrawler` further, e.g., to add more metadata to the index.

## Using a Master Crawler

A master crawler uses other crawlers to compile a combined master index of one or more data spaces.
This allows you and everyone else who has access to the master index, to search and possibly access all data within the shared data space.

To expose the project to a `MasterCrawler` we need to create a so called *access module*.
For signac projects this is simplified by using the `create_access_module()` method.
Let's create an access module:

In [ ]:
try:
    project.create_access_module({'.*/V\.txt': 'TextFile'})
except IOError:
    pass  # File already exists...

This function creates a file called `signac_access.py` within our project's root directory.

In [ ]:
% cat projects/tutorial/signac_access.py

You will notice that this file looks very similar to our custom crawler definition earlier.
It also shows us how to execute a Master Crawler for this data space.
Let's do that:

In [ ]:
from signac.contrib import MasterCrawler
master_crawler = MasterCrawler('projects')
master_index = list(master_crawler.crawl(depth=1))
for doc in master_index[:3]:
    print(doc)

The index generated by the master crawler contains all the information about our project, even the files, without any additional information.
This is possible, because the `MasterCrawler` searches the data space for files named `signac_access.py` and then collects all indexes generated by the `slave crawlers` defined within these modules.

This allows us to easily generate a *master index* of multiple projects and even directly fetch data, using only the index, see the next section.

## Fetch data via filename

Just like before, we can access data via the filenames specified in the index documents:

In [ ]:
import os

docs_files = [doc for doc in master_index if doc['format'] is not None]
for doc in docs_files[:3]:
    fn = os.path.join(doc['root'], doc['filename'])
    with open(fn) as file:
        print(doc['statepoint']['p'], file.read().strip())

## Fetch data via index documents

But even better, data files can be seamlessly fetched using the `signac.fetch()` function:

In [ ]:
for doc in docs_files[:3]:
    with signac.fetch(doc) as file:
        print(doc['statepoint']['p'], file.read().strip())

Think of `fetch()` like the built-in `open()` function. It allows us to retrieve and open files based on the index document (file id) instead of an absolute file path. This makes it easier to operate on data agnostic to its actual physical location.